# Document Clustering and Topic Modeling

*In* this project, we use unsupervised learning models to cluster unlabeled documents into different groups, visualize the results and identify their latent topics/structures.

**Things need to know**
* 对于0基础的同学，以了解为主
* 不需要可以从0把code写出来
* 这部分code与算法课程code的思维不一样
* 需要下课自己去跑一下code，尽量去理解
* 工具终归是辅助我们的，先尽力学会一个
* 逐步掌握python code debug能力

## Contents

* [Part 1: Load Data](#Part-1:-Load-Data)
* [Part 2: Tokenizing and Stemming](#Part-2:-Tokenizing-and-Stemming)
* [Part 3: TF-IDF](#Part-3:-TF-IDF)
* [Part 4: K-means clustering](#Part-4:-K-means-clustering)
* [Part 5: Topic Modeling - Latent Dirichlet Allocation](#Part-5:-Topic-Modeling---Latent-Dirichlet-Allocation)


# Part 0: Setup Google Drive Environment

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# https://drive.google.com/open?id=192JMR7SIqoa14vrs7Z9BXO3iK89pimJL
file = drive.CreateFile({'id':'192JMR7SIqoa14vrs7Z9BXO3iK89pimJL'}) # replace the id with id of file you want to access
file.GetContentFile('data.tsv')  

# Part 1: Load Data

In [0]:
import numpy as np
import pandas as pd
import nltk
import gensim
# REGULAR EXPRESSION
import re
import os

from sklearn import decomposition
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
# Load data into dataframe
df = pd.read_csv('data.tsv', sep='\t', header=0, error_bad_lines=False)

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

In [0]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [0]:
# Remove missing value
df.review_body.dropna(inplace=True)

In [0]:
# use the first 1000 data as our training data
data = df.loc[:1000, 'review_body'].tolist()

# Part 2: Tokenizing and Stemming

Load stopwords and stemmer function from NLTK library.
Stop words are words like "a", "the", or "in" which don't convey significant meaning.
Stemming is the process of breaking a word down into its root.

In [0]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english')

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

We use 179 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [0]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    # exclude stop words and tokenize the document, generate a list of string 
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word not in stopwords]

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

# tokenization without stemming
def tokenization(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word not in stopwords]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [0]:
# tokenization and stemming
tokenization_and_stemming(data[0])

['absolut',
 'love',
 'watch',
 'get',
 'compliment',
 'almost',
 'everi',
 'time',
 'i',
 'wear',
 'dainti']

Use our defined functions to analyze (i.e. tokenize, stem) our reviews.

In [0]:
# 1. do tokenization and stemming for all the documents
# 2. also just do tokenization for all the documents
# the goal is to create a mapping from stemmed words to original tokenized words for result interpretation.
docs_stemmed = []
docs_tokenized = []
for i in data:
    tokenized_and_stemmed_results = tokenization_and_stemming(i)
    docs_stemmed.extend(tokenized_and_stemmed_results)
    
    tokenized_results = tokenization(i)
    docs_tokenized.extend(tokenized_results)

In [0]:
# create a mapping from stemmed words to original words
vocab_frame_dict = {docs_stemmed[x]:docs_tokenized[x] for x in range(len(docs_stemmed))}

# Part 3: TF-IDF

TF: Term Frequency

IDF: Inverse Document Frequency

***example:***

document1: "Arthur da Jason"

document 2: "Jason da da huang"

document1: tf-idf [1, 0.5, 0.5, 0];  document2: tf-idf [0, 1, 0.5, 1]  

2-gram: 

document 1: Arthur da, da Jason; document 2: Jason da, da da, da huang bigram

3-gram:

document 1: Athur da Jason;  document 2: Jason da da, da da huang

[Arhur, da, Jason...]

In [0]:
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

In total, there are 1000 reviews and 245 terms.


In [0]:
# check the parameters
tfidf_model.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 0.99,
 'max_features': 1000,
 'min_df': 0.01,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': 'english',
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function __main__.tokenization_and_stemming>,
 'use_idf': True,
 'vocabulary': None}

Save the terms identified by TF-IDF.

In [0]:
# words
tf_selected_words = tfidf_model.get_feature_names()

In [0]:
# print out words
tf_selected_words

["'m",
 "'s",
 'abl',
 'absolut',
 'accur',
 'actual',
 'adjust',
 'alarm',
 'alreadi',
 'alway',
 'amaz',
 'amazon',
 'anoth',
 'arm',
 'arriv',
 'automat',
 'awesom',
 'bad',
 'band',
 'batteri',
 'beauti',
 'best',
 'better',
 'big',
 'bit',
 'black',
 'blue',
 'bought',
 'box',
 'br',
 'bracelet',
 'brand',
 'break',
 'bright',
 'broke',
 'button',
 'buy',
 'ca',
 'came',
 'case',
 'casio',
 'chang',
 'cheap',
 'clasp',
 'classi',
 'clock',
 'color',
 'come',
 'comfort',
 'compliment',
 'cool',
 'cost',
 'crown',
 'crystal',
 'dark',
 'date',
 'daughter',
 'day',
 'deal',
 'definit',
 'deliveri',
 'design',
 'dial',
 'differ',
 'difficult',
 'disappoint',
 'display',
 'dress',
 'durabl',
 'easi',
 'easili',
 'end',
 'everi',
 'everyday',
 'everyth',
 'exact',
 'excel',
 'expect',
 'expens',
 'face',
 'fair',
 'far',
 'fast',
 'featur',
 'feel',
 'fell',
 'fine',
 'finish',
 'fit',
 'function',
 'gave',
 'gift',
 'gold',
 'good',
 'got',
 'great',
 'hand',
 'happi',
 'hard',
 'heavi

# (Optional) Calculate Document Similarity

In [0]:
# use cosine similarity to check the similarity for two documents
from sklearn.metrics.pairwise import cosine_similarity
cos_matrix = cosine_similarity(tfidf_matrix)
print (cos_matrix)

[[1.         0.42500259 0.         ... 0.         0.0400497  0.        ]
 [0.42500259 1.         0.         ... 0.         0.09423401 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.0400497  0.09423401 0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


# Part 4: K-means clustering

In [0]:
# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

## 4.1. Analyze K-means Result

In [0]:
# create DataFrame films from all of the input files.
product = { 'review': df[:1000].product_title, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])

In [0]:
frame.head(10)

,review,cluster
0,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",0
1,Kenneth Cole New York Women's KC4944 Automatic...,0
2,Ritche 22mm Black Stainless Steel Bracelet Wat...,3
3,Citizen Men's BM8180-03E Eco-Drive Stainless S...,3
4,Orient ER27009B Men's Symphony Automatic Stain...,3
5,Casio Men's GW-9400BJ-1JF G-Shock Master of G ...,0
6,Fossil Women's ES3851 Urban Traveler Multifunc...,2
7,INFANTRY Mens Night Vision Analog Quartz Wrist...,3
8,G-Shock Men's Grey Sport Watch,2
9,Heiden Quad Watch Winder in Black Leather,3


In [0]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
3,630
0,132
2,99
1,76
4,63


In [0]:
print ("<Document clustering result by K-means>")

#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(vocab_frame_dict[tf_selected_words[ind]])
        print (vocab_frame_dict[tf_selected_words[ind]] + ",", end='')
    print ()
    
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")
    print (", ".join(cluster_reviews))
    print ()

<Document clustering result by K-means>
Cluster 0 words:loved,watch,beautiful,'s,wife,looks,
Cluster 0 reviews (132 reviews): 
Invicta Women's 15150 "Angel" 18k Yellow Gold Ion-Plated Stainless Steel and Brown Leather Watch, Kenneth Cole New York Women's KC4944 Automatic Silver Automatic Mesh Bracelet Analog Watch, Casio Men's GW-9400BJ-1JF G-Shock Master of G Rangeman Digital Solar Black Carbon Fiber Insert Watch, Casio - G-Shock - Gulfmaster - Black - GWN1000C-1A, Domire Fashion Accessories Trial Order New Quartz Fashion Weave Wrap Around Leather Bracelet Lady Woman Butterfly Wrist Watch, Casio Men's Slim Solar Multi-Function Analog-Digital Watch, Batman Kids' BAT4072 Black Rubber Batman Logo Strap Watch, Timex Easy Reader Day-Date Leather Strap Watch, Casio F108WH Water Resistant Digital Blue Resin Strap Watch, Stuhrling Original Women's 956.02 Symphony Gold-Tone Watch with Brown Genuine Leather Band, Seiko Men's SNKK27 Seiko 5 Stainless Steel Automatic Watch, Swiss Legend Women's 1

# Part 5: Topic Modeling - Latent Dirichlet Allocation

In [0]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5, learning_method = 'online')

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
# LDA requires integer values
tfidf_model_lda = CountVectorizer(max_df=0.99, max_features=500,
                                 min_df=0.01, stop_words='english',
                                 tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix_lda = tfidf_model_lda.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix_lda.shape[0]) + \
      " reviews and " + str(tfidf_matrix_lda.shape[1]) + " terms.")

In total, there are 1000 reviews and 245 terms.


In [0]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix_lda)
print(lda_output.shape)
print(lda_output)

(1000, 5)
[[0.02515191 0.02570744 0.63943405 0.28441354 0.02529306]
 [0.05091054 0.05166879 0.7944627  0.05189205 0.05106591]
 [0.2        0.2        0.2        0.2        0.2       ]
 ...
 [0.59922623 0.10059285 0.10003584 0.10014478 0.1000003 ]
 [0.05002168 0.0513821  0.79744814 0.05090355 0.05024454]
 [0.03403592 0.03349406 0.37875161 0.03384295 0.51987546]]


In [0]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 245)
[[ 0.20440958 20.50926369  0.20323623 ...  0.20807169  0.20707495
   0.20850504]
 [ 2.79454108  3.34687777  0.20370336 ...  0.44078482  0.45525509
  13.00207738]
 [23.12423285 92.17988647  4.95095615 ...  5.32799091 37.38234059
  23.23982252]
 [11.37788658 67.9044541  10.76640114 ...  1.95877832  9.60172056
  14.99471539]
 [10.65637277 48.56744685  0.25476987 ...  9.96919295 23.45654348
   4.08413886]]


In [0]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(data))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.03,0.03,0.64,0.28,0.03,2
Doc1,0.05,0.05,0.79,0.05,0.05,2
Doc2,0.20,0.20,0.20,0.20,0.20,0
Doc3,0.03,0.43,0.03,0.48,0.03,3
Doc4,0.08,0.33,0.15,0.43,0.01,3
Doc5,0.04,0.43,0.44,0.04,0.04,2
Doc6,0.03,0.60,0.31,0.03,0.03,1
Doc7,0.03,0.03,0.44,0.47,0.03,3
Doc8,0.01,0.01,0.95,0.01,0.01,2
Doc9,0.02,0.33,0.02,0.60,0.02,3


In [0]:
df_document_topic['topic'].value_counts().to_frame()

,topic
2,283
0,215
1,214
3,184
4,104


In [0]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model_lda.get_feature_names()
df_topic_words.index = topic_names

df_topic_words.head()

[[ 0.20440958 20.50926369  0.20323623 ...  0.20807169  0.20707495
   0.20850504]
 [ 2.79454108  3.34687777  0.20370336 ...  0.44078482  0.45525509
  13.00207738]
 [23.12423285 92.17988647  4.95095615 ...  5.32799091 37.38234059
  23.23982252]
 [11.37788658 67.9044541  10.76640114 ...  1.95877832  9.60172056
  14.99471539]
 [10.65637277 48.56744685  0.25476987 ...  9.96919295 23.45654348
   4.08413886]]


,'m,'s,abl,absolut,accur,actual,adjust,alarm,alreadi,alway,amaz,amazon,anoth,arm,arriv,automat,awesom,bad,band,batteri,beauti,best,better,big,bit,black,blue,bought,box,br,bracelet,brand,break,bright,broke,button,buy,ca,came,case,...,star,start,stop,strap,sturdi,style,stylish,super,sure,surpris,swim,tell,thank,thing,think,thought,time,timex,tini,tri,turn,use,valu,ve,want,watch,water,way,wear,week,weight,went,wife,wind,wish,work,worn,worth,wrist,year
Topic0,0.204410,20.509264,0.203236,0.206561,1.342552,0.205374,0.205201,1.664260,0.226833,0.258121,0.203052,0.202555,0.204868,0.207280,6.981253,0.202237,0.202996,0.206369,0.703956,8.856399,0.237020,0.525893,0.203149,0.306694,0.214649,0.206842,0.203830,1.450032,0.208755,0.206839,0.209112,0.214641,1.530964,0.204223,0.210939,0.217764,0.212728,0.222645,0.680312,0.225494,...,2.361789,0.202032,0.213021,0.205115,0.204562,0.202714,0.203657,0.216697,0.206588,0.206103,0.216355,2.880642,17.814440,0.202873,4.780395,7.906665,17.699372,0.210483,0.201618,0.202136,0.201989,10.986206,18.030623,0.208886,0.246047,5.756484,8.599282,0.238447,0.301699,2.464704,0.201788,0.206339,0.204356,0.202829,0.218507,52.768253,0.201272,0.208072,0.207075,0.208505
Topic1,2.794541,3.346878,0.203703,5.719576,0.499813,3.553965,0.245561,2.454826,0.209108,3.639452,13.784889,1.571421,0.579164,0.224862,2.382613,0.233281,23.646232,13.454801,11.496503,21.456683,17.370098,0.209099,13.487028,2.361429,0.203972,0.204388,0.213970,0.280585,1.523515,0.269725,0.204186,8.141801,0.217082,0.202621,25.214501,0.209050,3.735494,0.741203,21.965434,1.623920,...,4.777063,0.221191,12.614396,0.205542,9.497519,0.206694,0.766331,0.212045,0.325696,0.207914,0.204138,5.300754,0.204665,4.522543,2.937152,0.210263,24.319619,0.230609,0.202098,8.118522,0.202887,11.963509,0.216213,2.636048,0.211863,215.204288,0.206561,7.523434,0.904819,22.257579,4.810994,0.208984,0.203325,0.209321,0.213471,35.970436,0.211346,0.440785,0.455255,13.002077
Topic2,23.124233,92.179886,4.950956,0.235564,3.370386,9.383724,0.221898,5.678379,12.927762,3.366676,2.062120,0.836103,27.984253,0.257494,1.716546,2.769129,0.203543,1.601945,65.507855,1.778157,20.041114,0.683267,0.229597,7.602401,0.205046,10.837263,29.101233,15.256614,5.834781,0.285355,0.221634,4.335799,0.220600,5.143364,0.221154,1.698790,48.788605,18.159421,0.380169,4.099069,...,2.193760,0.204674,0.260812,29.592818,0.206754,26.723190,19.814751,10.157637,7.492958,0.226827,13.316861,13.360436,0.355486,10.293487,27.279386,7.973216,34.980452,12.116799,0.204290,0.240569,4.549052,15.238868,0.263695,10.306525,34.069342,343.075936,10.748552,4.874279,63.691448,2.884254,0.245779,4.782282,18.409268,0.204843,0.241162,16.721272,3.246776,5.327991,37.382341,23.239823
Topic3,11.377887,67.904454,10.766401,10.829499,7.589224,0.213671,3.071255,0.209183,0.212712,6.223669,4.254486,24.106986,0.346030,0.213502,10.401944,12.512754,0.202409,0.212327,99.175040,14.995541,40.115214,0.213052,12.881357,12.158399,0.216677,8.516610,0.550663,31.987025,5.072191,246.507192,3.737980,0.374959,13.105602,3.814951,0.210720,4.677870,7.629196,7.257721,2.829353,19.686723,...,21.918365,14.141738,14.454301,13.035329,1.818085,0.214860,0.213534,4.426270,13.032738,12.050261,0.204761,1.842483,4.313698,9.744585,2.249262,0.243833,90.277874,9.090374,0.208694,13.184353,0.223829,22.814189,3.367271,25.654850,8.935607,311.124563,11.656059,4.046849,29.021801,21.663817,0.207840,8.314524,0.206020,13.010915,11.235634,25.174502,0.216410,1.958778,9.601721,14.994715
Topic4,10.656373,48.567447,0.254770,0.445576,0.246724,3.106162,20.776360,7.547113,0.205034,0.269239,0.204481,0.291886,0.202985,15.404888,0.675120,1.838731,0.200944,9.993730,9.195795,0.436437,1.781201,15.803438,3.395812,39.529836,23.659250,6.535142,0.217639,0.222042,3.030553,0.624464,10.042303,0.233764,0.205863,1.543939,0.204542,10.397120,0.522223,0.233617,7.593536,5.715081,...,0.315422,0.862903,0.860444,5.594191,0.206426,4.533083,0.407250,0.212513,0.468198,0.230434,0.204516,0.207500,0.207083,0.214353,0.220741,0.344338,22.80150

In [0]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model_lda, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,good,nice,work,product,like,look,excel,expect,face,'s,valu,great,thank,time,fast
Topic 1,watch,great,price,look,qualiti,love,good,work,far,gift,broke,purchas,time,awesom,recommend
Topic 2,watch,love,n't,look,'s,great,band,color,wear,perfect,buy,light,realli,wrist,time
Topic 3,watch,br,band,time,'s,look,like,hand,n't,day,second,beauti,bought,month,wear
Topic 4,watch,'s,big,face,easi,littl,read,size,like,larg,easili,small,bit,wrist,time
